In [1]:
import sqlite3
import pandas as pd
import numpy as np

In [2]:
# Connect to (or create) the database file
conn = sqlite3.connect("company.db")
cursor = conn.cursor()

# # Create tables
cursor.execute("DROP TABLE IF EXISTS projects")
cursor.execute("DROP TABLE IF EXISTS employees")
cursor.execute("DROP TABLE IF EXISTS departments")

cursor.execute("""
CREATE TABLE departments (
    dept_id INTEGER PRIMARY KEY,
    dept_name TEXT NOT NULL
)
""")

cursor.execute("""
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    dept_id INTEGER,
    salary INTEGER,
    project_id INTEGER,
    FOREIGN KEY (dept_id) REFERENCES departments(dept_id),
    FOREIGN KEY (project_id) REFERENCES projects(project_id)
)
""")

cursor.execute("""
CREATE TABLE projects (
    project_id INTEGER PRIMARY KEY,
    project_name TEXT NOT NULL
)
""")

#This is optional but can be used to create an index that will be used to speed up data extraction from large datasets
#Indexes need to exist before you run the queries that will benefit from them
#because the query planner uses indexes at query time to decide how to optimize execution
cursor.execute("""
CREATE INDEX idx_salary ON employees(salary)
""")

# Insert sample data
departments_data = [(1, "Engineering"), (2, "Human Resources"), (3, "Computer Science"), (4, "Data Science"), (5, "Biology"), (6, "Mathematics")]
employees_data = []

names = ["Aaran", "Aaren", "Aarez", "Aarman", "Aaron", "Aaron-James", "Aarron", "Aaryan", "Aaryn", "Aayan", "Aazaan", "Abaan", "Abbas", "Abdallah", "Abdalroof", "Abdihakim", "Abdirahman", "Abdisalam", "Abdul", "Abdul-Aziz", "Abdulbasir", "Abdulkadir", "Abdulkarem", "Abdulkhader", "Abdullah", "Abdul-Majeed", "Abdulmalik", "Abdul-Rehman", "Abdur", "Abdurraheem", "Abdur-Rahman", "Abdur-Rehmaan", "Abel", "Abhinav", "Abhisumant", "Abid", "Abir", "Abraham", "Abu", "Abubakar", "Ace", "Adain", "Adam", "Adam-James", "Addison", "Addisson", "Adegbola", "Adegbolahan", "Aden", "Adenn", "Adie", "Adil", "Aditya", "Adnan", "Adrian", "Adrien", "Aedan", "Aedin", "Aedyn", "Aeron", "Afonso", "Ahmad", "Ahmed", "Ahmed-Aziz", "Ahoua", "Ahtasham", "Aiadan", "Aidan", "Aiden", "Aiden-Jack", "Aiden-Vee", "Aidian", "Aidy", "Ailin", "Aiman", "Ainsley", "Ainslie", "Airen", "Airidas", "Airlie", "AJ", "Ajay", "A-Jay", "Ajayraj", "Akan", "Akram", "Al", "Ala", "Alan", "Alanas", "Alasdair", "Alastair", "Alber", "Albert", "Albie", "Aldred", "Alec", "Aled", "Aleem", "Aleksandar", "Aleksander", "Aleksandr", "Aleksandrs", "Alekzander", "Alessandro", "Alessio", "Alex", "Alexander", "Alexei", "Alexx", "Alexzander", "Alf", "Alfee", "Alfie", "Alfred", "Alfy", "Alhaji", "Al-Hassan", "Ali", "Aliekber", "Alieu", "Alihaider", "Alisdair", "Alishan", "Alistair", "Alistar", "Alister", "Aliyaan", "Allan", "Allan-Laiton", "Allen", "Allesandro", "Allister", "Ally", "Alphonse", "Altyiab", "Alum", "Alvern", "Alvin", "Alyas", "Amaan", "Aman", "Amani", "Ambanimoh", "Ameer", "Amgad", "Ami", "Amin", "Amir", "Ammaar", "Ammar", "Ammer", "Amolpreet", "Amos", "Amrinder", "Amrit", "Amro", "Anay", "Andrea", "Andreas", "Andrei", "Andrejs", "Andrew", "Andy", "Anees", "Anesu", "Angel", "Angelo", "Angus", "Anir", "Anis", "Anish", "Anmolpreet", "Annan", "Anndra", "Anselm", "Anthony", "Anthony-John", "Antoine", "Anton", "Antoni", "Antonio", "Antony", "Antonyo", "Anubhav", "Aodhan", "Aon", "Aonghus", "Apisai", "Arafat", "Aran", "Arandeep", "Arann", "Aray", "Arayan", "Archibald", "Archie", "Arda", "Ardal", "Ardeshir", "Areeb", "Areez", "Aref", "Arfin", "Argyle", "Argyll", "Ari", "Aria", "Arian", "Arihant", "Aristomenis", "Aristotelis", "Arjuna", "Arlo", "Armaan", "Arman", "Armen", "Arnab", "Arnav", "Arnold", "Aron", "Aronas", "Arran", "Arrham", "Arron", "Arryn", "Arsalan", "Artem", "Arthur", "Artur", "Arturo", "Arun", "Arunas", "Arved", "Arya", "Aryan", "Aryankhan", "Aryian", "Aryn", "Asa", "Asfhan", "Ash", "Ashlee-jay", "Ashley", "Ashton", "Ashton-Lloyd", "Ashtyn", "Ashwin", "Asif", "Asim", "Aslam", "Asrar", "Ata", "Atal", "Atapattu", "Ateeq", "Athol", "Athon", "Athos-Carlos", "Atli", "Atom", "Attila", "Aulay", "Aun", "Austen", "Austin", "Avani", "Averon", "Avi", "Avinash", "Avraham", "Awais", "Awwal", "Axel", "Ayaan", "Ayan", "Aydan", "Ayden", "Aydin", "Aydon", "Ayman", "Ayomide", "Ayren", "Ayrton", "Aytug", "Ayub", "Ayyub", "Azaan", "Azedine", "Azeem", "Azim", "Aziz", "Azlan", "Azzam", "Azzedine", "Babatunmise", "Babur", "Bader", "Badr", "Badsha", "Bailee", "Bailey", "Bailie", "Bailley", "Baillie", "Baley", "Balian", "Banan", "Barath", "Barkley", "Barney", "Baron", "Barrie", "Barry", "Bartlomiej", "Bartosz", "Basher", "Basile", "Baxter", "Baye", "Bayley", "Beau", "Beinn", "Bekim", "Believe", "Ben", "Bendeguz", "Benedict", "Benjamin", "Benjamyn", "Benji", "Benn", "Bennett", "Benny", "Benoit", "Bentley", "Berkay", "Bernard", "Bertie", "Bevin", "Bezalel", "Bhaaldeen", "Bharath", "Bilal", "Bill", "Billy", "Binod", "Bjorn", "Blaike", "Blaine", "Blair", "Blaire", "Blake", "Blazej", "Blazey", "Blessing", "Blue", "Blyth", "Bo", "Boab", "Bob", "Bobby", "Bobby-Lee", "Bodhan", "Boedyn", "Bogdan", "Bohbi", "Bony", "Bowen", "Bowie", "Boyd", "Bracken", "Brad", "Bradan", "Braden", "Bradley", "Bradlie", "Bradly", "Brady", "Bradyn", "Braeden", "Braiden", "Brajan", "Brandan", "Branden", "Brandon", "Brandonlee", "Brandon-Lee", "Brandyn", "Brannan", "Brayden", "Braydon", "Braydyn", "Breandan", "Brehme", "Brendan", "Brendon", "Brendyn", "Breogan", "Bret", "Brett", "Briaddon", "Brian", "Brodi", "Brodie", "Brody", "Brogan", "Broghan", "Brooke", "Brooklin", "Brooklyn", "Bruce", "Bruin", "Bruno", "Brunon", "Bryan", "Bryce", "Bryden", "Brydon", "Brydon-Craig", "Bryn", "Brynmor", "Bryson", "Buddy", "Bully", "Burak", "Burhan", "Butali", "Butchi", "Byron", "Cabhan", "Cadan", "Cade", "Caden", "Cadon", "Cadyn", "Caedan", "Caedyn", "Cael", "Caelan", "Caelen", "Caethan", "Cahl", "Cahlum", "Cai", "Caidan", "Caiden", "Caiden-Paul", "Caidyn", "Caie", "Cailaen", "Cailean", "Caileb-John", "Cailin", "Cain", "Caine", "Cairn", "Cal", "Calan", "Calder", "Cale", "Calean", "Caleb", "Calen", "Caley", "Calib", "Calin", "Callahan", "Callan", "Callan-Adam", "Calley", "Callie", "Callin", "Callum", "Callun", "Callyn", "Calum", "Calum-James", "Calvin", "Cambell", "Camerin", "Cameron", "Campbel", "Campbell", "Camron", "Caolain", "Caolan", "Carl", "Carlo", "Carlos", "Carrich", "Carrick", "Carson", "Carter", "Carwyn", "Casey", "Casper", "Cassy", "Cathal", "Cator", "Cavan", "Cayden", "Cayden-Robert", "Cayden-Tiamo", "Ceejay", "Ceilan", "Ceiran", "Ceirin", "Ceiron", "Cejay", "Celik", "Cephas", "Cesar", "Cesare", "Chad", "Chaitanya", "Chang-Ha", "Charles", "Charley", "Charlie", "Charly", "Chase", "Che", "Chester", "Chevy", "Chi", "Chibudom", "Chidera", "Chimsom", "Chin", "Chintu", "Chiqal", "Chiron", "Chris", "Chris-Daniel", "Chrismedi", "Christian", "Christie", "Christoph", "Christopher", "Christopher-Lee", "Christy", "Chu", "Chukwuemeka", "Cian", "Ciann", "Ciar", "Ciaran", "Ciarian", "Cieran", "Cillian", "Cillin", "Cinar", "CJ", "C-Jay", "Clark", "Clarke", "Clayton", "Clement", "Clifford", "Clyde", "Cobain", "Coban", "Coben", "Cobi", "Cobie", "Coby", "Codey", "Codi", "Codie", "Cody", "Cody-Lee", "Coel", "Cohan", "Cohen", "Colby", "Cole", "Colin", "Coll", "Colm", "Colt", "Colton", "Colum", "Colvin", "Comghan", "Conal", "Conall", "Conan", "Conar", "Conghaile", "Conlan", "Conley", "Conli", "Conlin", "Conlly", "Conlon", "Conlyn", "Connal", "Connall", "Connan", "Connar", "Connel", "Connell", "Conner", "Connolly", "Connor", "Connor-David", "Conor", "Conrad", "Cooper", "Copeland", "Coray", "Corben", "Corbin", "Corey", "Corey-James", "Corey-Jay", "Cori", "Corie", "Corin", "Cormac", "Cormack", "Cormak", "Corran", "Corrie", "Cory", "Cosmo", "Coupar", "Craig", "Craig-James", "Crawford", "Creag", "Crispin", "Cristian", "Crombie", "Cruiz", "Cruz", "Cuillin", "Cullen", "Cullin", "Curtis", "Cyrus", "Daanyaal", "Daegan", "Daegyu", "Dafydd", "Dagon", "Dailey", "Daimhin", "Daithi", "Dakota", "Daksh", "Dale", "Dalong", "Dalton", "Damian", "Damien", "Damon", "Dan", "Danar", "Dane", "Danial", "Daniel", "Daniele", "Daniel-James", "Daniels", "Daniil", "Danish", "Daniyal", "Danniel", "Danny", "Dante", "Danyal", "Danyil", "Danys", "Daood", "Dara", "Darach", "Daragh", "Darcy", "D'arcy", "Dareh", "Daren", "Darien", "Darius", "Darl", "Darn", "Darrach", "Darragh", "Darrel", "Darrell", "Darren", "Darrie", "Darrius", "Darroch", "Darryl", "Darryn", "Darwyn", "Daryl", "Daryn", "Daud", "Daumantas", "Davi", "David", "David-Jay", "David-Lee", "Davie", "Davis", "Davy", "Dawid", "Dawson", "Dawud", "Dayem", "Daymian", "Deacon", "Deagan", "Dean", "Deano", "Decklan", "Declain", "Declan", "Declyan", "Declyn", "Dedeniseoluwa", "Deecan", "Deegan", "Deelan", "Deklain-Jaimes", "Del", "Demetrius", "Denis", "Deniss", "Dennan", "Dennin", "Dennis", "Denny", "Dennys", "Denon", "Denton", "Denver", "Denzel", "Deon", "Derek", "Derick", "Derin", "Dermot", "Derren", "Derrie", "Derrin", "Derron", "Derry", "Derryn", "Deryn", "Deshawn", "Desmond", "Dev", "Devan", "Devin", "Devlin", "Devlyn", "Devon", "Devrin", "Devyn", "Dex", "Dexter", "Dhani", "Dharam", "Dhavid", "Dhyia", "Diarmaid", "Diarmid", "Diarmuid", "Didier", "Diego", "Diesel", "Diesil", "Digby", "Dilan", "Dilano", "Dillan", "Dillon", "Dilraj", "Dimitri", "Dinaras", "Dion", "Dissanayake", "Dmitri", "Doire", "Dolan", "Domanic", "Domenico", "Domhnall", "Dominic", "Dominick", "Dominik", "Donald", "Donnacha", "Donnie", "Dorian", "Dougal", "Douglas", "Dougray", "Drakeo", "Dre", "Dregan", "Drew", "Dugald", "Duncan", "Duriel", "Dustin", "Dylan", "Dylan-Jack", "Dylan-James", "Dylan-John", "Dylan-Patrick", "Dylin", "Dyllan", "Dyllan-James", "Dyllon", "Eadie", "Eagann", "Eamon", "Eamonn", "Eason", "Eassan", "Easton", "Ebow", "Ed", "Eddie", "Eden", "Ediomi", "Edison", "Eduardo", "Eduards", "Edward", "Edwin", "Edwyn", "Eesa", "Efan", "Efe", "Ege", "Ehsan", "Ehsen", "Eiddon", "Eidhan", "Eihli", "Eimantas", "Eisa", "Eli", "Elias", "Elijah", "Eliot", "Elisau", "Eljay", "Eljon", "Elliot", "Elliott", "Ellis", "Ellisandro", "Elshan", "Elvin", "Elyan", "Emanuel", "Emerson", "Emil", "Emile", "Emir", "Emlyn", "Emmanuel", "Emmet", "Eng", "Eniola", "Enis", "Ennis", "Enrico", "Enrique", "Enzo", "Eoghain", "Eoghan", "Eoin", "Eonan", "Erdehan", "Eren", "Erencem", "Eric", "Ericlee", "Erik", "Eriz", "Ernie-Jacks", "Eroni", "Eryk", "Eshan", "Essa", "Esteban", "Ethan", "Etienne", "Etinosa", "Euan", "Eugene", "Evan", "Evann", "Ewan", "Ewen", "Ewing", "Exodi", "Ezekiel", "Ezra", "Fabian", "Fahad", "Faheem", "Faisal", "Faizaan", "Famara", "Fares", "Farhaan", "Farhan", "Farren", "Farzad", "Fauzaan", "Favour", "Fawaz", "Fawkes", "Faysal", "Fearghus", "Feden", "Felix", "Fergal", "Fergie", "Fergus", "Ferre", "Fezaan", "Fiachra", "Fikret", "Filip", "Filippo", "Finan", "Findlay", "Findlay-James", "Findlie", "Finlay", "Finley", "Finn", "Finnan", "Finnean", "Finnen", "Finnlay", "Finnley", "Fintan", "Fionn", "Firaaz", "Fletcher", "Flint", "Florin", "Flyn", "Flynn", "Fodeba", "Folarinwa", "Forbes", "Forgan", "Forrest", "Fox", "Francesco", "Francis", "Francisco", "Franciszek", "Franco", "Frank", "Frankie", "Franklin", "Franko", "Fraser", "Frazer", "Fred", "Freddie", "Frederick", "Fruin", "Fyfe", "Fyn", "Fynlay", "Fynn", "Gabriel", "Gallagher", "Gareth", "Garren", "Garrett", "Garry", "Gary", "Gavin", "Gavin-Lee", "Gene", "Geoff", "Geoffrey", "Geomer", "Geordan", "Geordie", "George", "Georgia", "Georgy", "Gerard", "Ghyll", "Giacomo", "Gian", "Giancarlo", "Gianluca", "Gianmarco", "Gideon", "Gil", "Gio", "Girijan", "Girius", "Gjan", "Glascott", "Glen", "Glenn", "Gordon", "Grady", "Graeme", "Graham", "Grahame", "Grant", "Grayson", "Greg", "Gregor", "Gregory", "Greig", "Griffin", "Griffyn", "Grzegorz", "Guang", "Guerin", "Guillaume", "Gurardass", "Gurdeep", "Gursees", "Gurthar", "Gurveer", "Gurwinder", "Gus", "Gustav", "Guthrie", "Guy", "Gytis", "Habeeb", "Hadji", "Hadyn", "Hagun", "Haiden", "Haider", "Hamad", "Hamid", "Hamish", "Hamza", "Hamzah", "Han", "Hansen", "Hao", "Hareem", "Hari", "Harikrishna", "Haris", "Harish", "Harjeevan", "Harjyot", "Harlee", "Harleigh", "Harley", "Harman", "Harnek", "Harold", "Haroon", "Harper", "Harri", "Harrington", "Harris", "Harrison", "Harry", "Harvey", "Harvie", "Harvinder", "Hasan", "Haseeb", "Hashem", "Hashim", "Hassan", "Hassanali", "Hately", "Havila", "Hayden", "Haydn", "Haydon", "Haydyn", "Hcen", "Hector", "Heddle", "Heidar", "Heini", "Hendri", "Henri", "Henry", "Herbert", "Heyden", "Hiro", "Hirvaansh", "Hishaam", "Hogan", "Honey", "Hong", "Hope", "Hopkin", "Hosea", "Howard", "Howie", "Hristomir", "Hubert", "Hugh", "Hugo", "Humza", "Hunter", "Husnain", "Hussain", "Hussan", "Hussnain", "Hussnan", "Hyden", "I", "Iagan", "Iain", "Ian", "Ibraheem", "Ibrahim", "Idahosa", "Idrees", "Idris", "Iestyn", "Ieuan", "Igor", "Ihtisham", "Ijay", "Ikechukwu", "Ikemsinachukwu", "Ilyaas", "Ilyas", "Iman", "Immanuel", "Inan", "Indy", "Ines", "Innes", "Ioannis", "Ireayomide", "Ireoluwa", "Irvin", "Irvine", "Isa", "Isaa", "Isaac", "Isaiah", "Isak", "Isher", "Ishwar", "Isimeli", "Isira", "Ismaeel", "Ismail", "Israel", "Issiaka", "Ivan", "Ivar", "Izaak", "J", "Jaay", "Jac", "Jace", "Jack", "Jacki", "Jackie", "Jack-James", "Jackson", "Jacky", "Jacob", "Jacques", "Jad", "Jaden", "Jadon", "Jadyn", "Jae", "Jagat", "Jago", "Jaheim", "Jahid", "Jahy", "Jai", "Jaida", "Jaiden", "Jaidyn", "Jaii", "Jaime", "Jai-Rajaram", "Jaise", "Jak", "Jake", "Jakey", "Jakob", "Jaksyn", "Jakub", "Jamaal", "Jamal", "Jameel", "Jameil", "James", "James-Paul", "Jamey", "Jamie", "Jan", "Jaosha", "Jardine", "Jared", "Jarell", "Jarl", "Jarno", "Jarred", "Jarvi", "Jasey-Jay", "Jasim", "Jaskaran", "Jason", "Jasper", "Jaxon", "Jaxson", "Jay", "Jaydan", "Jayden", "Jayden-James", "Jayden-Lee", "Jayden-Paul", "Jayden-Thomas", "Jaydn", "Jaydon", "Jaydyn", "Jayhan", "Jay-Jay", "Jayke", "Jaymie", "Jayse", "Jayson", "Jaz", "Jazeb", "Jazib", "Jazz", "Jean", "Jean-Lewis", "Jean-Pierre", "Jebadiah", "Jed", "Jedd", "Jedidiah", "Jeemie", "Jeevan", "Jeffrey", "Jensen", "Jenson", "Jensyn", "Jeremy", "Jerome", "Jeronimo", "Jerrick", "Jerry", "Jesse", "Jesuseun", "Jeswin", "Jevan", "Jeyun", "Jez", "Jia", "Jian", "Jiao", "Jimmy", "Jincheng", "JJ", "Joaquin", "Joash", "Jock", "Jody", "Joe", "Joeddy", "Joel", "Joey", "Joey-Jack", "Johann", "Johannes", "Johansson", "John", "Johnathan", "Johndean", "Johnjay", "John-Michael", "Johnnie", "Johnny", "Johnpaul", "John-Paul", "John-Scott", "Johnson", "Jole", "Jomuel", "Jon", "Jonah", "Jonatan", "Jonathan", "Jonathon", "Jonny", "Jonothan", "Jon-Paul", "Jonson", "Joojo", "Jordan", "Jordi", "Jordon", "Jordy", "Jordyn", "Jorge", "Joris", "Jorryn", "Josan", "Josef", "Joseph", "Josese", "Josh", "Joshiah", "Joshua", "Josiah", "Joss", "Jostelle", "Joynul", "Juan", "Jubin", "Judah", "Jude", "Jules", "Julian", "Julien", "Jun", "Junior", "Jura", "Justan", "Justin", "Justinas", "Kaan", "Kabeer", "Kabir", "Kacey", "Kacper", "Kade", "Kaden", "Kadin", "Kadyn", "Kaeden", "Kael", "Kaelan", "Kaelin", "Kaelum", "Kai", "Kaid", "Kaidan", "Kaiden", "Kaidinn", "Kaidyn", "Kaileb", "Kailin", "Kain", "Kaine", "Kainin", "Kainui", "Kairn", "Kaison", "Kaiwen", "Kajally", "Kajetan", "Kalani", "Kale", "Kaleb", "Kaleem", "Kal-el", "Kalen", "Kalin", "Kallan", "Kallin", "Kalum", "Kalvin", "Kalvyn", "Kameron", "Kames", "Kamil", "Kamran", "Kamron", "Kane", "Karam", "Karamvir", "Karandeep", "Kareem", "Karim", "Karimas", "Karl", "Karol", "Karson", "Karsyn", "Karthikeya", "Kasey", "Kash", "Kashif", "Kasim", "Kasper", "Kasra", "Kavin", "Kayam", "Kaydan", "Kayden", "Kaydin", "Kaydn", "Kaydyn", "Kaydyne", "Kayleb", "Kaylem", "Kaylum", "Kayne", "Kaywan", "Kealan", "Kealon", "Kean", "Keane", "Kearney", "Keatin", "Keaton", "Keavan", "Keayn", "Kedrick", "Keegan", "Keelan", "Keelin", "Keeman", "Keenan", "Keenan-Lee", "Keeton", "Kehinde", "Keigan", "Keilan", "Keir", "Keiran", "Keiren", "Keiron", "Keiryn", "Keison", "Keith", "Keivlin", "Kelam", "Kelan", "Kellan", "Kellen", "Kelso", "Kelum", "Kelvan", "Kelvin", "Ken", "Kenan", "Kendall", "Kendyn", "Kenlin", "Kenneth", "Kensey", "Kenton", "Kenyon", "Kenzeigh", "Kenzi", "Kenzie", "Kenzo", "Kenzy", "Keo", "Ker", "Kern", "Kerr", "Kevan", "Kevin", "Kevyn", "Kez", "Khai", "Khalan", "Khaleel", "Khaya", "Khevien", "Khizar", "Khizer", "Kia", "Kian", "Kian-James", "Kiaran", "Kiarash", "Kie", "Kiefer", "Kiegan", "Kienan", "Kier", "Kieran", "Kieran-Scott", "Kieren", "Kierin", "Kiern", "Kieron", "Kieryn", "Kile", "Killian", "Kimi", "Kingston", "Kinneil", "Kinnon", "Kinsey", "Kiran", "Kirk", "Kirwin", "Kit", "Kiya", "Kiyonari", "Kjae", "Klein", "Klevis", "Kobe", "Kobi", "Koby", "Koddi", "Koden", "Kodi", "Kodie", "Kody", "Kofi", "Kogan", "Kohen", "Kole", "Konan", "Konar", "Konnor", "Konrad", "Koray", "Korben", "Korbyn", "Korey", "Kori", "Korrin", "Kory", "Koushik", "Kris", "Krish", "Krishan", "Kriss", "Kristian", "Kristin", "Kristofer", "Kristoffer", "Kristopher", "Kruz", "Krzysiek", "Krzysztof", "Ksawery", "Ksawier", "Kuba", "Kurt", "Kurtis", "Kurtis-Jae", "Kyaan", "Kyan", "Kyde", "Kyden", "Kye", "Kyel", "Kyhran", "Kyie", "Kylan", "Kylar", "Kyle", "Kyle-Derek", "Kylian", "Kym", "Kynan", "Kyral", "Kyran", "Kyren", "Kyrillos", "Kyro", "Kyron", "Kyrran", "Lachlainn", "Lachlan", "Lachlann", "Lael", "Lagan", "Laird", "Laison", "Lakshya", "Lance", "Lancelot", "Landon", "Lang", "Lasse", "Latif", "Lauchlan", "Lauchlin", "Laughlan", "Lauren", "Laurence", "Laurie", "Lawlyn", "Lawrence", "Lawrie", "Lawson", "Layne", "Layton", "Lee", "Leigh", "Leigham", "Leighton", "Leilan", "Leiten", "Leithen", "Leland", "Lenin", "Lennan", "Lennen", "Lennex", "Lennon", "Lennox", "Lenny", "Leno", "Lenon", "Lenyn", "Leo", "Leon", "Leonard", "Leonardas", "Leonardo", "Lepeng", "Leroy", "Leven", "Levi", "Levon", "Levy", "Lewie", "Lewin", "Lewis", "Lex", "Leydon", "Leyland", "Leylann", "Leyton", "Liall", "Liam", "Liam-Stephen", "Limo", "Lincoln", "Lincoln-John", "Lincon", "Linden", "Linton", "Lionel", "Lisandro", "Litrell", "Liyonela-Elam", "LLeyton", "Lliam", "Lloyd", "Lloyde", "Loche", "Lochlan", "Lochlann", "Lochlan-Oliver", "Lock", "Lockey", "Logan", "Logann", "Logan-Rhys", "Loghan", "Lokesh", "Loki", "Lomond", "Lorcan", "Lorenz", "Lorenzo", "Lorne", "Loudon", "Loui", "Louie", "Louis", "Loukas", "Lovell", "Luc", "Luca", "Lucais", "Lucas", "Lucca", "Lucian", "Luciano", "Lucien", "Lucus", "Luic", "Luis", "Luk", "Luka", "Lukas", "Lukasz", "Luke", "Lukmaan", "Luqman", "Lyall", "Lyle", "Lyndsay", "Lysander", "Maanav", "Maaz", "Mac", "Macallum", "Macaulay", "Macauley", "Macaully", "Machlan", "Maciej", "Mack", "Mackenzie", "Mackenzy", "Mackie", "Macsen", "Macy", "Madaki", "Maddison", "Maddox", "Madison", "Madison-Jake", "Madox", "Mael", "Magnus", "Mahan", "Mahdi", "Mahmoud", "Maias", "Maison", "Maisum", "Maitlind", "Majid", "Makensie", "Makenzie", "Makin", "Maksim", "Maksymilian", "Malachai", "Malachi", "Malachy", "Malakai", "Malakhy", "Malcolm", "Malik", "Malikye", "Malo", "Ma'moon", "Manas", "Maneet", "Manmohan", "Manolo", "Manson", "Mantej", "Manuel", "Manus", "Marc", "Marc-Anthony", "Marcel", "Marcello", "Marcin", "Marco", "Marcos", "Marcous", "Marcquis", "Marcus", "Mario", "Marios", "Marius", "Mark", "Marko", "Markus", "Marley", "Marlin", "Marlon", "Maros", "Marshall", "Martin", "Marty", "Martyn", "Marvellous", "Marvin", "Marwan", "Maryk", "Marzuq", "Mashhood", "Mason", "Mason-Jay", "Masood", "Masson", "Matas", "Matej", "Mateusz", "Mathew", "Mathias", "Mathu", "Mathuyan", "Mati", "Matt", "Matteo", "Matthew", "Matthew-William", "Matthias", "Max", "Maxim", "Maximilian", "Maximillian", "Maximus", "Maxwell", "Maxx", "Mayeul", "Mayson", "Mazin", "Mcbride", "McCaulley", "McKade", "McKauley", "McKay", "McKenzie", "McLay", "Meftah", "Mehmet", "Mehraz", "Meko", "Melville", "Meshach", "Meyzhward", "Micah", "Michael", "Michael-Alexander", "Michael-James", "Michal", "Michat", "Micheal", "Michee", "Mickey", "Miguel", "Mika", "Mikael", "Mikee", "Mikey", "Mikhail", "Mikolaj", "Miles", "Millar", "Miller", "Milo", "Milos", "Milosz", "Mir", "Mirza", "Mitch", "Mitchel", "Mitchell", "Moad", "Moayd", "Mobeen", "Modoulamin", "Modu", "Mohamad", "Mohamed", "Mohammad", "Mohammad-Bilal", "Mohammed", "Mohanad", "Mohd", "Momin", "Momooreoluwa", "Montague", "Montgomery", "Monty", "Moore", "Moosa", "Moray", "Morgan", "Morgyn", "Morris", "Morton", "Moshy", "Motade", "Moyes", "Msughter", "Mueez", "Muhamadjavad", "Muhammad", "Muhammed", "Muhsin", "Muir", "Munachi", "Muneeb", "Mungo", "Munir", "Munmair", "Munro", "Murdo", "Murray", "Murrough", "Murry", "Musa", "Musse", "Mustafa", "Mustapha", "Muzammil", "Muzzammil", "Mykie", "Myles", "Mylo", "Nabeel", "Nadeem", "Nader", "Nagib", "Naif", "Nairn", "Narvic", "Nash", "Nasser", "Nassir", "Natan", "Nate", "Nathan", "Nathanael", "Nathanial", "Nathaniel", "Nathan-Rae", "Nawfal", "Nayan", "Neco", "Neil", "Nelson", "Neo", "Neshawn", "Nevan", "Nevin", "Ngonidzashe", "Nial", "Niall", "Nicholas", "Nick", "Nickhill", "Nicki", "Nickson", "Nicky", "Nico", "Nicodemus", "Nicol", "Nicolae", "Nicolas", "Nidhish", "Nihaal", "Nihal", "Nikash", "Nikhil", "Niki", "Nikita", "Nikodem", "Nikolai", "Nikos", "Nilav", "Niraj", "Niro", "Niven", "Noah", "Noel", "Nolan", "Noor", "Norman", "Norrie", "Nuada", "Nyah", "Oakley", "Oban", "Obieluem", "Obosa", "Odhran", "Odin", "Odynn", "Ogheneochuko", "Ogheneruno", "Ohran", "Oilibhear", "Oisin", "Ojima-Ojo", "Okeoghene", "Olaf", "Ola-Oluwa", "Olaoluwapolorimi", "Ole", "Olie", "Oliver", "Olivier", "Oliwier", "Ollie", "Olurotimi", "Oluwadamilare", "Oluwadamiloju", "Oluwafemi", "Oluwafikunayomi", "Oluwalayomi", "Oluwatobiloba", "Oluwatoni", "Omar", "Omri", "Oran", "Orin", "Orlando", "Orley", "Orran", "Orrick", "Orrin", "Orson", "Oryn", "Oscar", "Osesenagha", "Oskar", "Ossian", "Oswald", "Otto", "Owain", "Owais", "Owen", "Owyn", "Oz", "Ozzy", "Pablo", "Pacey", "Padraig", "Paolo", "Pardeepraj", "Parkash", "Parker", "Pascoe", "Pasquale", "Patrick", "Patrick-John", "Patrikas", "Patryk", "Paul", "Pavit", "Pawel", "Pawlo", "Pearce", "Pearse", "Pearsen", "Pedram", "Pedro", "Peirce", "Peiyan", "Pele", "Peni", "Peregrine", "Peter", "Phani", "Philip", "Philippos", "Phinehas", "Phoenix", "Phoevos", "Pierce", "Pierre-Antoine", "Pieter", "Pietro", "Piotr", "Porter", "Prabhjoit", "Prabodhan", "Praise", "Pranav", "Pravin", "Precious", "Prentice", "Presley", "Preston", "Preston-Jay", "Prinay", "Prince", "Prithvi", "Promise", "Puneetpaul", "Pushkar", "Qasim", "Qirui", "Quinlan", "Quinn", "Radmiras", "Raees", "Raegan", "Rafael", "Rafal", "Rafferty", "Rafi", "Raheem", "Rahil", "Rahim", "Rahman", "Raith", "Raithin", "Raja", "Rajab-Ali", "Rajan", "Ralfs", "Ralph", "Ramanas", "Ramit", "Ramone", "Ramsay", "Ramsey", "Rana", "Ranolph", "Raphael", "Rasmus", "Rasul", "Raul", "Raunaq", "Ravin", "Ray", "Rayaan", "Rayan", "Rayane", "Rayden", "Rayhan", "Raymond", "Rayne", "Rayyan", "Raza", "Reace", "Reagan", "Reean", "Reece", "Reed", "Reegan", "Rees", "Reese", "Reeve", "Regan", "Regean", "Reggie", "Rehaan", "Rehan", "Reice", "Reid", "Reigan", "Reilly", "Reily", "Reis", "Reiss", "Remigiusz", "Remo", "Remy", "Ren", "Renars", "Reng", "Rennie", "Reno", "Reo", "Reuben", "Rexford", "Reynold", "Rhein", "Rheo", "Rhett", "Rheyden", "Rhian", "Rhoan", "Rholmark", "Rhoridh", "Rhuairidh", "Rhuan", "Rhuaridh", "Rhudi", "Rhy", "Rhyan", "Rhyley", "Rhyon", "Rhys", "Rhys-Bernard", "Rhyse", "Riach", "Rian", "Ricards", "Riccardo", "Ricco", "Rice", "Richard", "Richey", "Richie", "Ricky", "Rico", "Ridley", "Ridwan", "Rihab", "Rihan", "Rihards", "Rihonn", "Rikki", "Riley", "Rio", "Rioden", "Rishi", "Ritchie", "Rivan", "Riyadh", "Riyaj", "Roan", "Roark", "Roary", "Rob", "Robbi", "Robbie", "Robbie-lee", "Robby", "Robert", "Robert-Gordon", "Robertjohn", "Robi", "Robin", "Rocco", "Roddy", "Roderick", "Rodrigo", "Roen", "Rogan", "Roger", "Rohaan", "Rohan", "Rohin", "Rohit", "Rokas", "Roman", "Ronald", "Ronan", "Ronan-Benedict", "Ronin", "Ronnie", "Rooke", "Roray", "Rori", "Rorie", "Rory", "Roshan", "Ross", "Ross-Andrew", "Rossi", "Rowan", "Rowen", "Roy", "Ruadhan", "Ruaidhri", "Ruairi", "Ruairidh", "Ruan", "Ruaraidh", "Ruari", "Ruaridh", "Ruben", "Rubhan", "Rubin", "Rubyn", "Rudi", "Rudy", "Rufus", "Rui", "Ruo", "Rupert", "Ruslan", "Russel", "Russell", "Ryaan", "Ryan", "Ryan-Lee", "Ryden", "Ryder", "Ryese", "Ryhs", "Rylan", "Rylay", "Rylee", "Ryleigh", "Ryley", "Rylie", "Ryo", "Ryszard", "Saad", "Sabeen", "Sachkirat", "Saffi", "Saghun", "Sahaib", "Sahbian", "Sahil", "Saif", "Saifaddine", "Saim", "Sajid", "Sajjad", "Salahudin", "Salman", "Salter", "Salvador", "Sam", "Saman", "Samar", "Samarjit", "Samatar", "Sambrid", "Sameer", "Sami", "Samir", "Sami-Ullah", "Samual", "Samuel", "Samuela", "Samy", "Sanaullah", "Sandro", "Sandy", "Sanfur", "Sanjay", "Santiago", "Santino", "Satveer", "Saul", "Saunders", "Savin", "Sayad", "Sayeed", "Sayf", "Scot", "Scott", "Scott-Alexander", "Seaan", "Seamas", "Seamus", "Sean", "Seane", "Sean-James", "Sean-Paul", "Sean-Ray", "Seb", "Sebastian", "Sebastien", "Selasi", "Seonaidh", "Sephiroth", "Sergei", "Sergio", "Seth", "Sethu", "Seumas", "Shaarvin", "Shadow", "Shae", "Shahmir", "Shai", "Shane", "Shannon", "Sharland", "Sharoz", "Shaughn", "Shaun", "Shaunpaul", "Shaun-Paul", "Shaun-Thomas", "Shaurya", "Shaw", "Shawn", "Shawnpaul", "Shay", "Shayaan", "Shayan", "Shaye", "Shayne", "Shazil", "Shea", "Sheafan", "Sheigh", "Shenuk", "Sher", "Shergo", "Sheriff", "Sherwyn", "Shiloh", "Shiraz", "Shreeram", "Shreyas", "Shyam", "Siddhant", "Siddharth", "Sidharth", "Sidney", "Siergiej", "Silas", "Simon", "Sinai", "Skye", "Sofian", "Sohaib", "Sohail", "Soham", "Sohan", "Sol", "Solomon", "Sonneey", "Sonni", "Sonny", "Sorley", "Soul", "Spencer", "Spondon", "Stanislaw", "Stanley", "Stefan", "Stefano", "Stefin", "Stephen", "Stephenjunior", "Steve", "Steven", "Steven-lee", "Stevie", "Stewart", "Stewarty", "Strachan", "Struan", "Stuart", "Su", "Subhaan", "Sudais", "Suheyb", "Suilven", "Sukhi", "Sukhpal", "Sukhvir", "Sulayman", "Sullivan", "Sultan", "Sung", "Sunny", "Suraj", "Surien", "Sweyn", "Syed", "Sylvain", "Symon", "Szymon", "Tadd", "Taddy", "Tadhg", "Taegan", "Taegen", "Tai", "Tait", "Taiwo", "Talha", "Taliesin", "Talon", "Talorcan", "Tamar", "Tamiem", "Tammam", "Tanay", "Tane", "Tanner", "Tanvir", "Tanzeel", "Taonga", "Tarik", "Tariq-Jay", "Tate", "Taylan", "Taylar", "Tayler", "Taylor", "Taylor-Jay", "Taylor-Lee", "Tayo", "Tayyab", "Tayye", "Tayyib", "Teagan", "Tee", "Teejay", "Tee-jay", "Tegan", "Teighen", "Teiyib", "Te-Jay", "Temba", "Teo", "Teodor", "Teos", "Terry", "Teydren", "Theo", "Theodore", "Thiago", "Thierry", "Thom", "Thomas", "Thomas-Jay", "Thomson", "Thorben", "Thorfinn", "Thrinei", "Thumbiko", "Tiago", "Tian", "Tiarnan", "Tibet", "Tieran", "Tiernan", "Timothy", "Timucin", "Tiree", "Tisloh", "Titi", "Titus", "Tiylar", "TJ", "Tjay", "T-Jay", "Tobey", "Tobi", "Tobias", "Tobie", "Toby", "Todd", "Tokinaga", "Toluwalase", "Tom", "Tomas", "Tomasz", "Tommi-Lee", "Tommy", "Tomson", "Tony", "Torin", "Torquil", "Torran", "Torrin", "Torsten", "Trafford", "Trai", "Travis", "Tre", "Trent", "Trey", "Tristain", "Tristan", "Troy", "Tubagus", "Turki", "Turner", "Ty", "Ty-Alexander", "Tye", "Tyelor", "Tylar", "Tyler", "Tyler-James", "Tyler-Jay", "Tyllor", "Tylor", "Tymom", "Tymon", "Tymoteusz", "Tyra", "Tyree", "Tyrnan", "Tyrone", "Tyson", "Ubaid", "Ubayd", "Uchenna", "Uilleam", "Umair", "Umar", "Umer", "Umut", "Urban", "Uri", "Usman", "Uzair", "Uzayr", "Valen", "Valentin", "Valentino", "Valery", "Valo", "Vasyl", "Vedantsinh", "Veeran", "Victor", "Victory", "Vinay", "Vince", "Vincent", "Vincenzo", "Vinh", "Vinnie", "Vithujan", "Vladimir", "Vladislav", "Vrishin", "Vuyolwethu", "Wabuya", "Wai", "Walid", "Wallace", "Walter", "Waqaas", "Warkhas", "Warren", "Warrick", "Wasif", "Wayde", "Wayne", "Wei", "Wen", "Wesley", "Wesley-Scott", "Wiktor", "Wilkie", "Will", "William", "William-John", "Willum", "Wilson", "Windsor", "Wojciech", "Woyenbrakemi", "Wyatt", "Wylie", "Wynn", "Xabier", "Xander", "Xavier", "Xiao", "Xida", "Xin", "Xue", "Yadgor", "Yago", "Yahya", "Yakup", "Yang", "Yanick", "Yann", "Yannick", "Yaseen", "Yasin", "Yasir", "Yassin", "Yoji", "Yong", "Yoolgeun", "Yorgos", "Youcef", "Yousif", "Youssef", "Yu", "Yuanyu", "Yuri", "Yusef", "Yusuf", "Yves", "Zaaine", "Zaak", "Zac", "Zach", "Zachariah", "Zacharias", "Zacharie", "Zacharius", "Zachariya", "Zachary", "Zachary-Marc", "Zachery", "Zack", "Zackary", "Zaid", "Zain", "Zaine", "Zaineddine", "Zainedin", "Zak", "Zakaria", "Zakariya", "Zakary", "Zaki", "Zakir", "Zakk", "Zamaar", "Zander", "Zane", "Zarran", "Zayd", "Zayn", "Zayne", "Ze", "Zechariah", "Zeek", "Zeeshan", "Zeid", "Zein", "Zen", "Zendel", "Zenith", "Zennon", "Zeph", "Zerah", "Zhen", "Zhi", "Zhong", "Zhuo", "Zi", "Zidane", "Zijie", "Zinedine", "Zion", "Zishan", "Ziya", "Ziyaan", "Zohaib", "Zohair", "Zoubaeir", "Zubair", "Zubayr", "Zuriel"]
salary = ["60000","70000","80000","90000","100000","110000","120000"]
projects_data = [(1,"Research"),(2,"Training"),(3,"Cloud Migration"),(4,"Analysis"),(5,"Natural Language Processing")]
for x in range(0,1000):
    dept_id = int(np.random.uniform(0,6,1)[0])#No one is in the mathematics department, to include it change to this (0,7,1)
    name_index = int(np.random.uniform(0,2738,1)[0])
    age = int(np.random.uniform(19,97,1)[0])
    salary_index = int(np.random.uniform(0,7,1)[0])
    emp_proj_id = int(np.random.uniform(0,5,1)[0])
    emp_value = [x+1,names[name_index],age,dept_id,salary[salary_index],emp_proj_id]
    employees_data.append(emp_value)
    

print(employees_data[:20])
print("\n\n")
print(projects_data)

cursor.executemany("INSERT INTO departments VALUES (?, ?)", departments_data)
cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?, ?, ?)", employees_data)
cursor.executemany("INSERT INTO projects VALUES (?, ?)", projects_data)

conn.commit()



[[1, 'Benedict', 77, 5, '100000', 4], [2, 'Reace', 80, 1, '60000', 3], [3, 'Macallum', 90, 2, '90000', 0], [4, 'Strachan', 51, 0, '110000', 2], [5, 'Jensyn', 48, 0, '100000', 3], [6, 'Eduardo', 46, 3, '120000', 0], [7, 'Luca', 88, 3, '90000', 2], [8, 'Asif', 58, 4, '60000', 3], [9, 'Ridwan', 57, 3, '120000', 1], [10, 'Taliesin', 59, 0, '90000', 0], [11, 'Jaydyn', 28, 2, '80000', 3], [12, 'Favour', 80, 4, '60000', 4], [13, 'Baron', 83, 5, '80000', 1], [14, 'Leyton', 40, 2, '100000', 1], [15, 'Kyhran', 77, 3, '110000', 2], [16, 'Timothy', 67, 3, '90000', 1], [17, 'Shay', 71, 2, '120000', 4], [18, 'Issiaka', 94, 5, '110000', 3], [19, 'Aslam', 73, 1, '60000', 3], [20, 'Cesare', 31, 5, '70000', 4]]



[(1, 'Research'), (2, 'Training'), (3, 'Cloud Migration'), (4, 'Analysis'), (5, 'Natural Language Processing')]


In [3]:
# Show all employees
df_employees = pd.read_sql_query("SELECT * FROM employees", conn)
print("Employees Table:")
print(df_employees)

# Join employees with departments
query = """
SELECT e.name, e.salary, d.dept_name
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
"""
df_join = pd.read_sql_query(query, conn)
print("\nEmployees with Departments:")
print(df_join)

# List projects and their assigned employees
query = """
SELECT p.project_name, e.name
FROM employees e
JOIN projects p ON e.project_id = p.project_id
"""
df_projects = pd.read_sql_query(query, conn)
print("\nProjects with Employees:")
print(df_projects)

#list all employees in the Engineering Department
query = """
SELECT d.dept_name, e.name, e.salary
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
WHERE d.dept_name = 'Engineering'
"""
df_ens = pd.read_sql_query(query, conn)
print("The Engineering Salaries")
print(df_ens)

#This command will tell us what tables are in employees
# cursor.execute("PRAGMA table_info(employees)")
# print(cursor.fetchall())

#list all employees in the Engineering Department with their Research and then select the top 5
query = """
SELECT e.name, d.dept_name, e.salary, p.project_name
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
JOIN projects p ON e.dept_id = p.project_id
WHERE d.dept_name = 'Engineering' AND p.project_name = 'Research'
ORDER BY e.salary DESC LIMIT 5
"""
df_ens = pd.read_sql_query(query, conn)
print("The Engineering Salaries with their Research Areas")
print(df_ens)


#Show the 10 youngest employees(based on age), including their department name
query = """
SELECT e.name AS Name, d.dept_name AS Department, e.age AS Age
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
ORDER BY e.age ASC
LIMIT 10
"""
df_highemp = pd.read_sql_query(query, conn)
print("\n\nThe 10 youngest employees")
print(df_highemp)


#list the top 3 projects with the highest number of employees
query = """
SELECT p.project_name, COUNT(*) AS employee_count
FROM employees e
JOIN projects p ON e.project_id = p.project_id
GROUP BY p.project_name
ORDER BY employee_count DESC
LIMIT 3 
"""
df_highemp = pd.read_sql_query(query, conn)
print("\nProjects with the highest number of employees")
print(df_highemp)

#list all the departments in alphabetical order
query = """
SELECT d.dept_name
FROM departments d
ORDER BY d.dept_name ASC
"""
df_highemp = pd.read_sql_query(query, conn)
print("\n\nDepartments in alphabetical order")
print(df_highemp)

#list the top 2 earners in Engineering
query = """
SELECT e.name, e.salary
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
WHERE d.dept_name = 'Engineering'
ORDER BY e.salary DESC
LIMIT 2
"""
df_highemp = pd.read_sql_query(query, conn)
print("\n\nTop 2 highest earners in Engineering")
print(df_highemp)

#list all the departments in alphabetical order
query = """
SELECT e.name, e.age, e.salary
FROM employees e
ORDER BY RANDOM()
LIMIT 1
"""
df_highemp = pd.read_sql_query(query, conn)
print("\n\nDepartments in alphabetical order")
print(df_highemp)

#list all the departments by the number of employees from high to low
query = """
SELECT d.dept_name, COUNT(e.emp_id) AS num_employees
FROM employees e
JOIN departments d ON d.dept_id = e.dept_id
GROUP BY d.dept_name
ORDER BY num_employees DESC
"""
df_highemp = pd.read_sql_query(query, conn)
print("\n\nDepartments by employees in alphabetical order")
print(df_highemp)


Employees Table:
     emp_id      name  age  dept_id  salary  project_id
0         1  Benedict   77        5  100000           4
1         2     Reace   80        1   60000           3
2         3  Macallum   90        2   90000           0
3         4  Strachan   51        0  110000           2
4         5    Jensyn   48        0  100000           3
..      ...       ...  ...      ...     ...         ...
995     996      Yves   19        4   70000           0
996     997    Reigan   55        3   70000           0
997     998     Malik   73        1  100000           1
998     999    Taegen   53        4   80000           0
999    1000  Jesuseun   66        0   90000           2

[1000 rows x 6 columns]

Employees with Departments:
         name  salary         dept_name
0    Benedict  100000           Biology
1       Reace   60000       Engineering
2    Macallum   90000   Human Resources
3     Eduardo  120000  Computer Science
4        Luca   90000  Computer Science
..        ...    

In [4]:
#Find the names and salaries of all employees who are in Data Science department 
query = """
SELECT e.name, e.salary, d.dept_name
FROM employees e
JOIN Departments d ON d.dept_id = e.dept_id
WHERE d.dept_name = 'Data Science'
"""

df = pd.read_sql_query(query, conn)
print("\n\n These are the names and salaries of all employees in the Data Science department")
print(df)

#Show the names, salary and department ID of all employees who earn more than 80000
query = """
SELECT name, salary, dept_id
FROM employees
WHERE salary > 80000
"""

df = pd.read_sql_query(query, conn)
print("\n\nThese are the names, salaries and departments of all employees who earn more than 80000")
print(df)

query = """
SELECT name, salary
FROM employees
WHERE salary <= 60000
"""

df = pd.read_sql_query(query, conn)
print("\n\nThese are the names and salaries of all employees who earn less than 60000 or who earn 60000")
print(df)

query = """
SELECT name, salary
FROM employees
WHERE name LIKE 'J%'
"""

df = pd.read_sql_query(query, conn)
print("\n\nThese are the names and salary of all employees whose name starts with J")
print(df)

query = """
SELECT name, salary
FROM employees
WHERE name LIKE '%son%'
"""

df = pd.read_sql_query(query, conn)
print("\n\nThese are the names and salaries of all employees whose name contain 'son'")
print(df)

query = """
SELECT name, salary, dept_id
FROM employees
WHERE dept_id=1 OR dept_id=3 OR dept_id=5
"""

df = pd.read_sql_query(query, conn)
print("\n\nThese are the names and salaries of all employees who are in department 1, 3 or 5")
print(df)

query = """
SELECT name, salary, dept_id
FROM employees
WHERE dept_id!=2 AND dept_id!=4 
"""

df = pd.read_sql_query(query, conn)
print("\n\nThese are the names and salaries of all employees who are not in department 2 or 4")
print(df)

query = """
SELECT name, salary
FROM employees
WHERE salary BETWEEN 60000 AND 90001
"""

df = pd.read_sql_query(query, conn)
print("\n\nThese are the names and salary of all employees who earn between 60000 and 90000(Inclusive)")
print(df)

query = """
SELECT name, salary, dept_id
FROM employees
WHERE salary = 70000 AND dept_id = 1
"""

df = pd.read_sql_query(query, conn)
print("\n\nThese are the names, salary and department id of employees who earn 70000 and are in department 1")
print(df)

query = """
SELECT name, project_id
FROM employees
WHERE project_id IS NOT NULL
"""

df = pd.read_sql_query(query, conn)
print("\n\nThese are the names and project_id of employees who are not in a project")
print(df)





 These are the names and salaries of all employees in the Data Science department
          name  salary     dept_name
0         Asif   60000  Data Science
1       Favour   60000  Data Science
2        Deryn  120000  Data Science
3         Amro   60000  Data Science
4    Nicodemus   80000  Data Science
..         ...     ...           ...
159      Maxim  100000  Data Science
160     Clarke   80000  Data Science
161    Quinlan   80000  Data Science
162       Yves   70000  Data Science
163     Taegen   80000  Data Science

[164 rows x 3 columns]


These are the names, salaries and departments of all employees who earn more than 80000
         name  salary  dept_id
0    Macallum   90000        2
1        Luca   90000        3
2    Taliesin   90000        0
3     Timothy   90000        3
4      Karsyn   90000        2
..        ...     ...      ...
586    Joshua  120000        0
587    Kenyon  120000        2
588   Lyndsay  120000        1
589      Adil  120000        1
590     Blair  12

In [5]:
#Joining Operations
#By default INNER JOIN and JOIN are the same
query = """
SELECT e.name, e.salary, d.dept_name
FROM employees e
INNER JOIN departments d ON e.dept_id = d.dept_id
"""

df_join = pd.read_sql_query(query, conn)
print("\n\nThe names of employees in departments")
print(df_join)

query = """
SELECT e.name, e.salary, p.project_name
FROM employees e
JOIN projects p ON e.project_id = p.project_id
"""

df_join = pd.read_sql_query(query, conn)
print("\n\nThe names of employees working on projects")
print(df_join)

query = """
SELECT e.name, d.dept_name
FROM employees e
LEFT JOIN departments d ON e.dept_id = d.dept_id
"""

df_join = pd.read_sql_query(query, conn)
print("\n\nThe names and departments of employees")
print(df_join)

query = """
SELECT e.name, p.project_name
FROM employees e
LEFT JOIN projects p ON e.project_id = p.project_id
"""

df_join = pd.read_sql_query(query, conn)
print("\n\nThe names and projects of employees")
print(df_join)

query = """
SELECT d.dept_name, e.name
FROM departments d
LEFT JOIN employees e ON d.dept_id = e.dept_id
"""

df_join = pd.read_sql_query(query, conn)
print("\n\nThe departments with their employees")
print(df_join)

query = """
SELECT p.project_name, e.name
FROM projects p
LEFT JOIN employees e ON p.project_id = e.project_id
"""

df_join = pd.read_sql_query(query, conn)
print("\n\nThe projects with each employee working on them")
print(df_join)

query = """
SELECT e.name, d.dept_name
FROM employees e
FULL OUTER JOIN departments d ON e.dept_id = d.dept_id
"""

df_join = pd.read_sql_query(query, conn)
print("\n\nThe list of all employees and departments")
print(df_join)

query = """
SELECT e.name, p.project_name
FROM employees e
FULL OUTER JOIN projects p ON e.project_id = p.project_id
"""

df_join = pd.read_sql_query(query, conn)
print("\n\nThe list of all employees and projects")
print(df_join)

query = """
SELECT e.name, d.dept_name, p.project_name
FROM employees e
JOIN departments d ON e.dept_id = d.dept_id
JOIN projects p ON e.project_id = p.project_id
"""

df_join = pd.read_sql_query(query, conn)
print("\n\nThe names of all employees with projects and departments")
print(df_join)

query = """
SELECT e.name, d.dept_name, p.project_name
FROM employees e
LEFT JOIN departments d ON e.dept_id = d.dept_id
LEFT JOIN projects p ON e.project_id = p.project_id
"""

df_join = pd.read_sql_query(query, conn)
print("\n\nThe names of all employees, departments and projects")
print(df_join)




The names of employees in departments
         name  salary         dept_name
0    Benedict  100000           Biology
1       Reace   60000       Engineering
2    Macallum   90000   Human Resources
3     Eduardo  120000  Computer Science
4        Luca   90000  Computer Science
..        ...     ...               ...
840    Conner  100000  Computer Science
841      Yves   70000      Data Science
842    Reigan   70000  Computer Science
843     Malik  100000       Engineering
844    Taegen   80000      Data Science

[845 rows x 3 columns]


The names of employees working on projects
         name  salary     project_name
0    Benedict  100000         Analysis
1       Reace   60000  Cloud Migration
2    Strachan  110000         Training
3      Jensyn  100000  Cloud Migration
4        Luca   90000         Training
..        ...     ...              ...
782  Kenzeigh   70000         Analysis
783   Quinlan   80000         Training
784    Conner  100000         Training
785     Malik  100000

In [6]:
print("\nQuestion 1")
query = """
SELECT COUNT(*) AS Total_Employees
FROM employees e
"""

df_sql = pd.read_sql_query(query, conn)
print("\nThe total number of employees")
print(df_sql)

print("\nQuestion 2")
query = """
SELECT AVG(salary) AS Average_Salary
FROM employees
"""

df_sql = pd.read_sql_query(query, conn)
print("\nThe average salary of all employees")
print(df_sql)

print("\nQuestion 3")
query = """
SELECT d.dept_name, AVG(e.salary)
FROM departments d
LEFT JOIN employees e ON d.dept_id = e.dept_id
GROUP BY d.dept_name
"""

df_sql = pd.read_sql_query(query, conn)
print("\nThe average salary for each department")
print(df_sql)

print("\nQuestion 4")
query = """
SELECT d.dept_name, MAX(e.salary)
FROM departments d
LEFT JOIN employees e ON d.dept_id = e.dept_id
GROUP BY d.dept_name
"""

df_sql = pd.read_sql_query(query, conn)
print("\nThe Highest salary in every department")
print(df_sql)

print("\nQuestion 5")
query = """
SELECT d.dept_name, COUNT(e.emp_id) AS Number_Of_Employees_In_Department
FROM departments d
LEFT JOIN employees e ON d.dept_id = e.dept_id
GROUP BY d.dept_name
HAVING COUNT(e.emp_id) > 5
"""

df_sql = pd.read_sql_query(query, conn)
print("\nThe name of departments with more than 5 employees")
print(df_sql)

print("\nQuestion 6")
query = """
SELECT p.project_name AS Project, AVG(e.salary) AS Avg_Salary
FROM projects p
LEFT JOIN employees e ON p.project_id = e.project_id
GROUP BY p.project_name
"""

df_sql = pd.read_sql_query(query, conn)
print("\nThe average salary per project")
print(df_sql)

print("\nQuestion 7")
query = """
SELECT p.project_name AS Project_Name, AVG(e.salary) AS salary
FROM projects p
LEFT JOIN employees e ON p.project_id = e.project_id
GROUP BY p.project_name
HAVING AVG(e.salary) > 60000
"""

df_sql = pd.read_sql_query(query, conn)
print("\nProjects with an average salary above 60000")
print(df_sql)

print("\nQuestion 8")
query = """
SELECT d.dept_name AS Department, AVG(e.salary) AS Salary
FROM departments d
JOIN employees e ON d.dept_id = e.dept_id
WHERE d.dept_name IS NOT NULL
GROUP BY d.dept_name
ORDER BY AVG(e.salary) ASC
LIMIT 1
"""

df_sql = pd.read_sql_query(query, conn)
print("\nThe department with the lowest average salary")
print(df_sql)

print("\nQuestion 9")
#How many employees are in each department and what is their salary
query = """
SELECT d.dept_name, COUNT(e.emp_id), SUM(e.salary)
FROM departments d
LEFT JOIN employees e ON d.dept_id = e.dept_id
GROUP BY d.dept_name
"""

df_sql = pd.read_sql_query(query, conn)
print("\nThe number of employees in each department along with their total salary")
print(df_sql)

print("\nQuestion 10")
query = """
SELECT d.dept_name
FROM departments d
LEFT JOIN employees e ON e.dept_id = d.dept_id
GROUP BY d.dept_name
HAVING MIN(e.salary) < 40000
"""

df_sql = pd.read_sql_query(query, conn)
print("\nThe department with employees that have a minimum salary below 40000")
print(df_sql)






Question 1

The total number of employees
   Total_Employees
0             1000

Question 2

The average salary of all employees
   Average_Salary
0         90710.0

Question 3

The average salary for each department
          dept_name  AVG(e.salary)
0           Biology   89894.179894
1  Computer Science   92732.919255
2      Data Science   90670.731707
3       Engineering   92335.329341
4   Human Resources   90914.634146
5       Mathematics            NaN

Question 4

The Highest salary in every department
          dept_name  MAX(e.salary)
0           Biology       120000.0
1  Computer Science       120000.0
2      Data Science       120000.0
3       Engineering       120000.0
4   Human Resources       120000.0
5       Mathematics            NaN

Question 5

The name of departments with more than 5 employees
          dept_name  Number_Of_Employees_In_Department
0           Biology                                189
1  Computer Science                                161
2      Data

In [7]:
#Nested SQL Queries and SubQueries
print("\nQuestion 1")
query = """
SELECT name, salary, (SELECT AVG(salary) FROM employees) AS Average_Salary
FROM employees
"""

df_sql = pd.read_sql_query(query, conn)
print("Each employees salary alongside the average salary in the company")
print(df_sql)

print("\nQuestion 2")
query = """
SELECT dept_name, (SELECT COUNT(emp_id) FROM employees) AS Total_Employee
FROM departments
"""

df_sql = pd.read_sql_query(query, conn)
print("Each departments name with the total number of employees in the company")
print(df_sql)

print("\nQuestion 3")
query = """
SELECT Name, Average
FROM (
    SELECT d.dept_name AS Name, AVG(e.salary) AS Average 
    FROM departments d 
    LEFT JOIN employees e ON d.dept_id = e.dept_id
    GROUP BY d.dept_name
    ) AS Average_Salary
WHERE Average > 90000
"""

df_sql = pd.read_sql_query(query, conn)
print("Departments with and average salary greater than 90000")
print(df_sql)

print("\nQuestion 4")
query = """
SELECT name, salary
FROM (
    SELECT d.dept_name AS name, AVG(e.salary) AS salary
    FROM departments d
    LEFT JOIN employees e ON d.dept_id = e.dept_id
    GROUP BY d.dept_name
) AS Top_Three
ORDER BY salary DESC
LIMIT 3
"""

df_sql = pd.read_sql_query(query, conn)
print("Top 3 departments with the highest average salary")
print(df_sql)

print("\nQuestion 5")
query = """
SELECT e.name
FROM employees e
WHERE e.dept_id = (
    SELECT dept_id
    FROM (
        SELECT d.dept_id, AVG(e.salary) AS avg_salary
        FROM departments d
        JOIN employees e ON d.dept_id = e.dept_id
        GROUP BY d.dept_id
        ORDER BY avg_salary DESC
        LIMIT 1
    ) AS top_dept
)
"""


df_sql = pd.read_sql_query(query, conn)
print("The employees who work in the department with the highest average salary")
print(df_sql)

print("\nQuestion 6")
query = """
SELECT project_name
FROM (
    SELECT p.project_name, AVG(e.salary) AS avg_salary
    FROM projects p
    JOIN employees e ON p.project_id = e.project_id
    GROUP BY p.project_name
) AS project_salaries
WHERE avg_salary > (
    SELECT AVG(salary)
    FROM employees
)
"""

df_sql = pd.read_sql_query(query, conn)
print("Projects that have employees whose average salary is greater than the overall average salary")
print(df_sql)

print("\nQuestion 7")
query = """
SELECT name
FROM employees
WHERE salary > (
    SELECT MAX(e.salary)
    FROM employees e
    JOIN departments d ON d.dept_id = e.dept_id
    WHERE d.dept_name = 'Data Science'
)
"""

df_sql = pd.read_sql_query(query, conn)
print("Employees whose salary is higher than the highest salary in the Data Science department")
print(df_sql)

#Find employees whose salary is higher than the average salary of their department.
query = """
SELECT e.name
FROM employees e
WHERE e.salary > (
    SELECT AVG(e2.salary)
    FROM employees e2
    WHERE e2.dept_id = e.dept_id
)
"""

df_sql = pd.read_sql_query(query, conn)
print("Employees whose salary is higher than the average salary of their department")
print(df_sql)

query = """
SELECT d.dept_name
FROM departments d
WHERE d.dept_id NOT IN (
    SELECT e.dept_id
    FROM employees e
    WHERE e.salary < 70000
)
"""

df_sql = pd.read_sql_query(query, conn)
print("departments where every employee earns more than 70,000")
print(df_sql)

# SELECT p.project_name, COUNT(e.emp_id) AS employee_count
# FROM projects p
# JOIN employees e ON p.project_id = e.project_id
# GROUP BY p.project_id
# HAVING COUNT(e.emp_id) > (
#     SELECT AVG(emp_count)
#     FROM (
#         SELECT COUNT(e2.emp_id) AS emp_count
#         FROM projects p2
#         JOIN employees e2 ON p2.project_id = e2.project_id
#         GROUP BY p2.project_id
#     ) AS project_counts
# )
query = """
SELECT project_name, employee_count
FROM (
    SELECT p.project_name, COUNT(e.emp_id) AS employee_count
    FROM projects p
    JOIN employees e ON p.project_id = e.project_id
    GROUP BY p.project_name
) AS project_employees
WHERE employee_count > (
    SELECT AVG(emp_count)
    FROM (
        SELECT COUNT(e2.emp_id) AS emp_count
        FROM projects p2
        JOIN employees e2 ON p2.project_id = e2.project_id
        GROUP BY p2.project_id
    ) AS avg_table
)
"""

df_sql = pd.read_sql_query(query, conn)
print("Projects where the number of employees is greater than the project average number of employees")
print(df_sql)

#Find employees who earn more than the second highest salary in the company
query = """
SELECT e2.name, e2.salary
FROM employees e2
WHERE e2.salary > (
    SELECT salary_list
    FROM(
        SELECT e.salary AS salary_list
        FROM employees e
        GROUP BY e.salary 
        ORDER BY e.salary DESC
        LIMIT 2
    ) AS SAL
    ORDER BY salary_list ASC
    LIMIT 1
)
"""

df_sql = pd.read_sql_query(query, conn)
print("Employees who earn more than the second highest salary in the company")
print(df_sql)

# 1. Find departments where the average salary is lower than the overall company average.
# 2. List employees who work on the project with the fewest employees.
# 3. Find projects that have more than one department involved.
# 4. List employees who earn the highest salary in their department.
# 5. Show departments where at least one employee earns below 60,000, but not all do.

query = """
SELECT DEPT, SAL
FROM (
    SELECT d.dept_name AS DEPT, AVG(e.salary) AS SAL
    FROM departments d
    JOIN employees e ON d.dept_id = e.emp_id
    GROUP BY d.dept_name
) AS DEPT_AVG
WHERE SAL < (
    SELECT AVG(e.salary)
    FROM employees e
)
"""

df_sql = pd.read_sql_query(query, conn)
print("The departments where the average salary is lower than the overall company average")
print(df_sql)

query = """
SELECT e2.name
FROM employees e2
JOIN projects p2 ON e2.project_id = p2.project_id
WHERE p2.project_name = (
    SELECT NAME
        FROM(
            SELECT p.project_name AS NAME, COUNT(e.emp_id) AS EMP_COUNT
            FROM projects p
            JOIN employees e ON p.project_id = e.project_id
            GROUP BY p.project_name
            ORDER BY EMP_COUNT ASC
            LIMIT 1
        )
)
"""

df_sql = pd.read_sql_query(query, conn)
print("List employees who work on the projects with the fewest employees")
print(df_sql)

query = """
SELECT e.name, e.salary, DEPT_MAX.Name
FROM employees e
JOIN (
    SELECT Name, SAL, ID
    FROM (
        SELECT d.dept_name AS Name, MAX(e.salary) AS SAL, d.dept_id AS ID
        FROM departments d
        JOIN employees e ON d.dept_id = e.dept_id
        GROUP BY d.dept_name
        )) AS DEPT_MAX ON e.dept_id = DEPT_MAX.ID
WHERE e.salary = DEPT_MAX.SAL
"""

df_sql = pd.read_sql_query(query, conn)
print("Employees who earn the highest salary in their department")
print(df_sql)

# 5. Show departments where at least one employee earns below 60,000, but not all do.
query = """
SELECT d.dept_name
FROM departments d
JOIN employees e ON d.dept_id = e.dept_id
GROUP BY d.dept_name
HAVING 
    SUM(CASE WHEN e.salary < 70000 THEN 1 ELSE 0 END) >= 1 AND
    SUM(CASE WHEN e.salary >= 70000 THEN 1 ELSE 0 END) >= 1
"""

df_sql = pd.read_sql_query(query, conn)
print("Departments where at least one employee earns below 60,000")
print(df_sql)




Question 1
Each employees salary alongside the average salary in the company
         name  salary  Average_Salary
0    Benedict  100000         90710.0
1       Reace   60000         90710.0
2    Macallum   90000         90710.0
3    Strachan  110000         90710.0
4      Jensyn  100000         90710.0
..        ...     ...             ...
995      Yves   70000         90710.0
996    Reigan   70000         90710.0
997     Malik  100000         90710.0
998    Taegen   80000         90710.0
999  Jesuseun   90000         90710.0

[1000 rows x 3 columns]

Question 2
Each departments name with the total number of employees in the company
          dept_name  Total_Employee
0       Engineering            1000
1   Human Resources            1000
2  Computer Science            1000
3      Data Science            1000
4           Biology            1000
5       Mathematics            1000

Question 3
Departments with and average salary greater than 90000
               Name       Average
0  C

In [8]:
#pd.options.display.max_rows = 9999 #This will increase the number of rows displayed to 999
#pd.set_option('display.float_format', lambda x: '%.2f' % x) #This will change the way floats are displayed, showing only 2 decimal places


#Examples of CASE Functions, Window Functions, and Common Table Expressions (CTE)
query = """
SELECT name, dept_id, salary,
RANK() OVER (PARTITION BY dept_id ORDER BY salary DESC) AS dept_rank
FROM employees
"""

df_sql = pd.read_sql_query(query, conn)
print("Example use case of Window Functions")
print(df_sql)

query = """
WITH DeptAvg AS (
    SELECT dept_id, AVG(salary) AS avg_salary
    FROM employees
    GROUP BY dept_id
)
SELECT e.name, e.salary, d.avg_salary
FROM employees e
JOIN DeptAvg d ON e.dept_id = d.dept_id
WHERE e.salary > d.avg_salary
"""

df_sql = pd.read_sql_query(query, conn)
print("Example using Common Table Expressions (CTE)")
print(df_sql)

query = """
SELECT name,
       salary,
       CASE 
           WHEN salary > 100000 THEN 'High'
           WHEN salary BETWEEN 70000 AND 100000 THEN 'Medium'
           ELSE 'Low'
       END AS salary_band
FROM employees
"""

df_sql = pd.read_sql_query(query, conn)
print("Example use case of Window Functions")
print(df_sql)

query = """
SELECT name, 
        CASE 
            WHEN salary >= 100000 THEN 'Excellent'
            WHEN salary >= 70000 AND salary < 100000 THEN 'Good'
            ELSE 'Need Improvement'
        END AS performance_category
FROM employees
"""

df_sql = pd.read_sql_query(query, conn)
print("New Category which categorizes all the salaries")
print(df_sql)

#List the top 2 highest paid employees per department, including their department ID, name, and salary.
query = """
SELECT Id, Name, salary, Part, dept_name
FROM (
    SELECT e.dept_id AS Id, e.name AS Name, e.salary AS Salary, ROW_NUMBER() OVER (PARTITION BY e.dept_id ORDER BY e.salary DESC) AS Part
    FROM employees e
    ) AS Partition
JOIN departments d ON Partition.Id = d.dept_id
WHERE Part = 1 OR Part = 2
"""


df_sql = pd.read_sql_query(query, conn)
print("The top 2 highest paid employees in each department")
print(df_sql)

#Show how the results of RANK() and ROW_NUMBER() differ when two or more employees in the same department have the same salary
query = """
SELECT e.name, e.salary, e.dept_id, RANK() OVER (PARTITION BY e.dept_id ORDER BY e.salary) AS rank, ROW_NUMBER() OVER (PARTITION BY e.dept_id ORDER BY e.salary) AS row_number
FROM employees e
"""

df_sql = pd.read_sql_query(query, conn)
print("\nRANK and ROW_NUMBER difference")
print(df_sql)

#Using a CTE, list employees whose salary is higher than the average salary of their department

query = """
WITH Dept_Avg AS (
SELECT dept_id, AVG (salary) AS average
FROM employees
GROUP BY dept_id
)

SELECT e.name, e.salary, d.average
FROM employees e
JOIN Dept_Avg d ON d.dept_id = e.dept_id
WHERE e.salary > d.average
"""

df_sql = pd.read_sql_query(query, conn)
print("The employees who earn more than their department average")
print(df_sql)

query = """
WITH Highest_Dept AS (
SELECT d.dept_name, d.dept_id, MAX(e.salary) AS max_salary
FROM departments d
JOIN employees e ON d.dept_id = e.dept_id
GROUP BY d.dept_name
)

SELECT e.name, e.salary, h.max_salary, h.dept_name, CASE WHEN salary > 100000 THEN 'Excellent' END AS salary_remark
FROM employees e
JOIN Highest_Dept h ON h.dept_id = e.dept_id
WHERE e.salary = h.max_salary
GROUP BY h.dept_name
"""

df_sql = pd.read_sql_query(query, conn)
print("\nThe highest paid employee in each department is")
print(df_sql)

query = """
WITH RankedEmployees AS (
    SELECT 
        e.name, 
        e.salary, 
        e.dept_id,
        d.dept_name,
        RANK() OVER (PARTITION BY e.dept_id ORDER BY e.salary DESC) AS rnk
    FROM employees e
    JOIN departments d ON e.dept_id = d.dept_id
),
SingleTopPerDept AS (
    SELECT 
        name,
        salary,
        dept_id,
        dept_name,
        rnk,
        CASE 
            WHEN salary >= 100000 THEN 'Excellent'
            WHEN salary >= 70000 THEN 'Good'
            ELSE 'Needs Improvement'
        END AS salary_remark,
        ROW_NUMBER() OVER (PARTITION BY dept_id ORDER BY salary DESC, name) AS row_num
    FROM RankedEmployees
    WHERE rnk = 1
)

SELECT name, salary, dept_name, salary_remark
FROM SingleTopPerDept
WHERE row_num = 1;
"""

df_sql = pd.read_sql_query(query, conn)
print("The highest paid employee in each department computed using multiple variables in ORDER BY ")
print(df_sql)



Example use case of Window Functions
         name  dept_id  salary  dept_rank
0         Chu        0  120000          1
1        Igor        0  120000          1
2      Peiyan        0  120000          1
3       Derin        0  120000          1
4      Girius        0  120000          1
..        ...      ...     ...        ...
995      Jiao        5   60000        166
996      Luke        5   60000        166
997    Harlee        5   60000        166
998     Kyaan        5   60000        166
999  Tristain        5   60000        166

[1000 rows x 4 columns]
Example using Common Table Expressions (CTE)
         name  salary    avg_salary
0    Benedict  100000  89894.179894
1    Strachan  110000  87677.419355
2      Jensyn  100000  87677.419355
3     Eduardo  120000  92732.919255
4      Ridwan  120000  92732.919255
..        ...     ...           ...
495  Ahtasham  100000  90914.634146
496     Blair  120000  89894.179894
497    Conner  100000  92732.919255
498     Malik  100000  92335.

In [9]:
#In a situation where we have many tables to join each we can use a for loop 
# joins = []
# for i in range(1, 101):
#     joins.append(f"JOIN table_{i} t{i} ON main_table.id = t{i}.main_id")

# query = f"""
# SELECT main_table.*
# FROM main_table
# {' '.join(joins)}
# """

In [10]:
#This is how you create a view and use it
# CREATE VIEW a_with_details AS
# SELECT a.*, b.col_b, c.col_c, d.col_d, e.col_e
# FROM a
# JOIN b ON a.b_id = b.id
# JOIN c ON a.c_id = c.id
# JOIN d ON a.d_id = d.id
# JOIN e ON a.e_id = e.id;

# SELECT * FROM a_with_details WHERE b.col_b = 'SomeValue';
